In [1]:
from torch import tensor, stack, float64, float32, profiler
from torch import device
import torch
import numpy as np
from hamiltonian import Sky_phi
from hamiltonian import ham_total
from xitorch import linalg
from test_sparse_eigen import CsrLinOp
from energy_gap import energy_gap
from adaptive.learner.learnerND import curvature_loss_function
from adaptive import notebook_extension, Runner, BlockingRunner, LearnerND, Learner2D
from tools.device_setter import multi_GPU_executor

In [ ]:
def main():
    try:
        torch.multiprocessing.set_start_method('spawn')
    except:
        pass

    print("Learner initialization")
    loss_function = curvature_loss_function()
    learner = LearnerND(energy_gap, bounds = [(0, 1), (-1, 1), (0.2, 1.25), (0.5, 3)], loss_per_simplex = loss_function)
    #learner = Learner2D(energy_gap, bounds = [(0, 1), (-1, 1)])
    print("Runner initialization")

    tasks_per_gpu = 1
    gpus = range(13,15)
    executor = multi_GPU_executor(tasks_per_gpu = tasks_per_gpu, gpus = gpus)

    runner = Runner(learner, executor = executor, shutdown_executor = True, retries = 2, goal = lambda l: l.npoints > 0)#l.loss() < 0.000001)
    if type(runner) != BlockingRunner:
        runner.live_info(update_interval = 1)

    return runner, learner

runner, learner = main()